In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
#@title 0. 공통 설정 (매번 실행)
from dotenv import load_dotenv
from pathlib import Path
import os

WORKING_DIR = "DataTraining"
WORKING_TREE = "/content/drive/MyDrive/ColabNotebooks/"+WORKING_DIR

load_dotenv(dotenv_path=Path(WORKING_TREE)/".env")

GIT_TOKEN = os.getenv("GIT_TOKEN")
svcKey = os.getenv("svcKey")

GIT_USERNAME = "seizethedayunhui"
GIT_REPO = "datatraining"
REPO = WORKING_TREE+"/"+GIT_REPO  # colab 밑 여러분이 작업하는 디렉토리. github 에 동기될 루트 저장소

%cd "{REPO}"

/content/drive/MyDrive/ColabNotebooks/DataTraining/datatraining


In [5]:
#@title 0. 로컬(colab) 한글 환경 설정
# 그래프에 한글을 표기하기 위한 설정 (폰트 설치 + ...)
!sudo apt-get update -qq
!sudo apt-get install -y fonts-nanum fonts-nanum-coding
!sudo fc-cache -fv
!rm -rf ~/.cache/matplotlib
!rm -rf /tmp/.cache/matplotlib

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  fonts-nanum fonts-nanum-coding
0 upgraded, 2 newly installed, 0 to remove and 58 not upgraded.
Need to get 10.3 MB of archives.
After this operation, 34.1 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 fonts-nanum all 20200506-1 [10.3 MB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 fonts-nanum-coding all 2.5-3 [4,988 B]
Fetched 10.3 MB in 1s (7,795 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 2.)
debconf: falling back to

In [6]:
#@title 1. API 데이터 수집 테스트
import requests

# url = 'https://apis.data.go.kr/B552584/EvCharger/getChargerInfo?serviceKey=여러분의_키&pageNo=1&numOfRows=10&zcode=26'
url = 'http://apis.data.go.kr/B552584/EvCharger/getChargerInfo'
# url = 'http://apis.data.go.kr/B552584/EvCharger/getChargerStatus'

# svcKey는 decoding key (encoding 하지 않은 key라는 뜻에서 decoding key. 실제로 원래 키 -> 인코딩 -> 디코딩 은 아님); 환경설정에 기술
# svcKey=""
params = {'serviceKey' : svcKey, 'pageNo' : '1', 'numOfRows' : '10', 'zcode' : '26' }  # zcode; 26=부산, 11=서울

response = requests.get(url, params=params)
print(response.content)


b'<?xml version="1.0" encoding="UTF-8" standalone="yes"?><response><header><resultCode>00</resultCode><resultMsg>NORMAL SERVICE.</resultMsg><totalCount>28393</totalCount><pageNo>1</pageNo><numOfRows>10</numOfRows></header><body><items><item><statNm>\xeb\xb6\x80\xec\x82\xb0 \xec\x96\xb4\xeb\xa6\xb0\xec\x9d\xb4 \xeb\x8c\x80\xea\xb3\xb5\xec\x9b\x90</statNm><statId>ME181207</statId><chgerId>01</chgerId><chgerType>06</chgerType><addr>\xeb\xb6\x80\xec\x82\xb0\xea\xb4\x91\xec\x97\xad\xec\x8b\x9c \xeb\xb6\x80\xec\x82\xb0\xec\xa7\x84\xea\xb5\xac \xec\x84\xb1\xec\xa7\x80\xea\xb3\xa1\xeb\xa1\x9c 3</addr><addrDetail>null</addrDetail><location>null</location><lat>35.1831134</lat><lng>129.0459801</lng><useTime>24\xec\x8b\x9c\xea\xb0\x84 \xec\x9d\xb4\xec\x9a\xa9\xea\xb0\x80\xeb\x8a\xa5</useTime><busiId>ME</busiId><bnm>\xed\x99\x98\xea\xb2\xbd\xeb\xb6\x80</bnm><busiNm>\xed\x99\x98\xea\xb2\xbd\xeb\xb6\x80</busiNm><busiCall>1661-9408</busiCall><stat>2</stat><statUpdDt>20251202204251</statUpdDt><lastTsdt

In [7]:
#@title 2. 데이터 가져오기 위한 준비_1
# zcode에 대해, pageNo, numOfRows 를 지정해서 데이터를 가져오는 API 밖에 정의되어 있지 않네...
import xml.etree.ElementTree as ET

# Update params for the initial request
params['pageNo'] = '1'
params['numOfRows'] = '9999'
params['zcode'] = '26'

# Make the API request
response = requests.get(url, params=params)

# Parse the XML response
root = ET.fromstring(response.content)

# Extract totalCount
total_count = int(root.find('header').find('totalCount').text)

# Extract all item elements
all_items = root.find('body').find('items').findall('item')

print(f"Total count for zcode=26: {total_count}")
print(f"Number of items in the first page (up to 9999): {len(all_items)}")

Total count for zcode=26: 28393
Number of items in the first page (up to 9999): 9999


In [8]:
#@title 3. 데이터 가져오기 위한 준비_2
# numOfRows를 최대치로 했는데 전부 못 가져왔으니, 나누어서 데이터를 가져와야겠네. 몇번에 나누어서 가져오면 좋을까?
import math

num_of_rows = int(params['numOfRows'])
total_pages = math.ceil(total_count / num_of_rows)

print(f"Total pages to retrieve all data for zcode=26: {total_pages}")

Total pages to retrieve all data for zcode=26: 3
